In [1]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json


from functions import (get_sector, changes_from_press, get_content_header,
                       get_df, normalise_to_index, save_data)

columns = ['date', 'press title', 'url', '1d change', 'index_price','norm_price']
past_last_date = False

In [2]:
data = []
page_num = 0

ticker = 'tsla'

sector = get_sector(ticker)

index_ticker = '^GSPC'
# select by value 

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)

# create break
while True:
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"}
    
    s = requests.Session()
    html = requests.get(f'https://ir.tesla.com/press?page={page_num}/', headers=headers)
    

    content = BeautifulSoup(html.content) 

    articles = content.find_all('div',attrs={'class': 'views-row'})
        
    if not articles:
        break
   
    for article in articles:
        date = article.find('time').text
        
        sub_art = article.find('h4', attrs={'class': 'press-release-teaser__title'})
        title = sub_art.find('a').text
        url = sub_art.find('a').get('href')
        url = f'https://ir.tesla.com{url}'
        content = get_content_header(url, 'meta', {'name':'twitter:description'})

        
        date = date.lstrip().rstrip()    # remove starting and trailing whitespaces
        title = title.lstrip().rstrip() 

        date = datetime.strptime(date, '%b %d, %Y').date()
        
        if date < datetime(2014, 1, 1).date():
            past_last_date = True
            break        
        
        pct_change = changes_from_press(stock_hist, date, 1)
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)

        data.append([date, title, url, pct_change, index_price, norm_price])
    if past_last_date == True: break    
    page_num = page_num + 1


save_data(data, ticker, sector)

TypeError: save_data() takes 3 positional arguments but 4 were given